# Setting up the inital dataframes

In [ ]:
import pandas as pd
# orders.csv
url = "https://drive.google.com/file/d/1Vu0q91qZw6lqhIqbjoXYvYAQTmVHh6uZ/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orders = pd.read_csv(path)

# orderlines.csv
url = "https://drive.google.com/file/d/1FYhN_2AzTBFuWcfHaRuKcuCE6CWXsWtG/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orderlines = pd.read_csv(path)

# products.csv
url = "https://drive.google.com/file/d/1afxwDXfl-7cQ_qLwyDitfcCx3u7WMvkU/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products = pd.read_csv(path)

products_initial = products.copy()

# ORDERS

There are no duplicated rows in orders.

There are 5 missing values in orders (represents 0.0022% of all rows).

`created_date` should become datetime datatype.

In [ ]:
orders = orders.loc[~orders.total_paid.isna(), :] # simply delete these 5 rows, as we have enough data without them
orders["created_date"] = pd.to_datetime(orders["created_date"])
orders_cl = orders.copy()

<ipython-input-9-4abae6f7ac9c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders["created_date"] = pd.to_datetime(orders["created_date"])


# ORDERLINES

There are no duplicated rows in orderlines.

There are no missing values in orderlines.

Optional: `product_id` is not in use and can be deleted.

Column `date` should be a datetime datatype.


In [ ]:
# orderlines.drop(columns='product_id', inplace=True)
orderlines["date"] = pd.to_datetime(orderlines["date"])

Column `unit_price` should be a float datatype. Initially, it has string datatype and some values includes 2 dots, therefore datatype can't be changed simply with `pd.to_numeric`

## DROP: drop all rows with 2 dots in `unit_price` and all connected orders.

In [ ]:
two_dot_order_ids_list = orderlines.loc[orderlines.unit_price.str.contains("\d+\.\d+\.\d+"), "id_order"]
orderlines_drop = orderlines.loc[~orderlines.id_order.isin(two_dot_order_ids_list)].copy()
orderlines_drop["unit_price"] = pd.to_numeric(orderlines_drop["unit_price"])

## CORRECT: remove extra dots in values and save the affected rows.

In [ ]:
orderlines_correct = orderlines.copy()
orderlines_correct.loc[orderlines_correct.unit_price.str.count('\.') > 1, 'unit_price'] = (orderlines_correct
                                                                                .loc[orderlines_correct.unit_price
                                                                                .str.count('\.') > 1, 'unit_price']
                                                                                .str.replace(".", "" , 1)
                                                                                )
orderlines_correct["unit_price"] = pd.to_numeric(orderlines_correct["unit_price"])

<ipython-input-15-5966e439296c>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  .str.replace(".", "" , 1)


*Choosing* DROP or CORRECT (start one of the code lines below)

In [ ]:
orderlines_cl = orderlines_drop.copy()

In [ ]:
orderlines_cl = orderlines_correct.copy()

# PRODUCTS

In [ ]:
f'{(products.duplicated().sum()/products.shape[0]*100).round(3)}% of all rows are duplicates'

In [ ]:
products.drop_duplicates(inplace=True)

Rows with NaN values stands for 0.90737% of total amount of unique rows. Dropping them

In [ ]:
products = products.loc[~(products.price.isna() | products.type.isna())]